# InceptionResNetV2

## Get embeddings
I just download this model from Keras and inference it following their example

In [1]:
# Load images
import numpy as np
from PIL import Image
import os

# Training
image_folder = "images/training_imgs"

# Break into categories
clothing_images = [Image.open(os.path.join(image_folder, file)) for file in os.listdir(image_folder) if file.endswith("c.png")]
clothing_labels = [file.split("-")[0].split("_") for file in os.listdir(image_folder) if file.endswith("c.png")]
clothing_labels = [[lab[0], lab[1], lab[2][:-1]] if lab[2].endswith("s") else lab for lab in clothing_labels]

full_body_images = [Image.open(os.path.join(image_folder, file)) for file in os.listdir(image_folder) if file.endswith("fb.png")]
full_body_labels = [file.split("-")[0].split("_") for file in os.listdir(image_folder) if file.endswith("fb.png")]
full_body_filenames = [file for file in os.listdir(image_folder) if file.endswith("fb.png")]
full_body_labels = [[lab[0], lab[1], lab[2][:-1]] if lab[2].endswith("s") else lab for lab in full_body_labels]

# # Test
# image_folder = "images/example_imgs"

# # Break into categories
# test_clothing_images = [Image.open(os.path.join(image_folder, file)) for file in os.listdir(image_folder) if file.endswith("c.png")]
# test_clothing_labels = [file.split("-")[0].split("_") for file in os.listdir(image_folder) if file.endswith("c.png")]
# test_clothing_labels = [[lab[0], lab[1], lab[2][:-1]] if lab[2].endswith("s") else lab for lab in clothing_labels]

In [ ]:
# Create embeddings
import tensorflow as tf

def sample_generator(dataset):
    for sample in dataset:
        yield tf.keras.applications.vgg19.inception_resnet_v2(np.array(sample))

img_spec = tf.TensorSpec((488, 488, 3), float)
ds_clothing = tf.data.Dataset.from_generator(lambda: sample_generator(clothing_images), output_signature=img_spec).batch(32)
ds_outfit = tf.data.Dataset.from_generator(lambda: sample_generator(full_body_images), output_signature=img_spec).batch(32)
# ds_test = tf.data.Dataset.from_generator(lambda: sample_generator(test_clothing_images), output_signature=img_spec).batch(32)

model = tf.keras.applications.InceptionResNetV2(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=(488, 488, 3),
    pooling="avg"
)

clothing_embed = model.predict(ds_clothing)
full_body_embed = model.predict(ds_outfit)
# test_embed = model.predict(ds_test)

2024-05-05 02:11:17.425472: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-05 02:11:17.472086: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-05 02:11:18.127030: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-05-05 02:11:19.041809: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 920 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:18:00.0, compute capab

## Visualize model embeddings

In [ ]:
# Load images
from pickle import load
import numpy as np
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

embeds = load(open("jepa_encodings_train.pkl", "rb"))

# Break into categories
clothing_pca = PCA(n_components=2).fit(clothing_embed)
print(clothing_pca.explained_variance_ratio_, sum(clothing_pca.explained_variance_ratio_))
clothing_pca = clothing_pca.transform(clothing_embed)
clothing_tsne = TSNE().fit_transform(clothing_embed)

full_body_pca = PCA(n_components=2).fit(full_body_embed)
print(full_body_pca.explained_variance_ratio_, sum(full_body_pca.explained_variance_ratio_))
full_body_pca = full_body_pca.transform(full_body_embed)
full_body_tsne = TSNE().fit_transform(full_body_embed)

In [ ]:
# Plot gendered items
import matplotlib.pyplot as plt

for gender in ["mens", "womens"]:
    plt.scatter(
        clothing_pca[[i for i in range(len(clothing_pca)) if clothing_labels[i][1] == gender]][:, 0],
        clothing_pca[[i for i in range(len(clothing_pca)) if clothing_labels[i][1] == gender]][:, 1],
        label=gender
    )

plt.title("PCA Representation of Clothes of Different Clothing Genders")
plt.tight_layout()
plt.legend()
plt.show()

for gender in ["mens", "womens"]:
    plt.scatter(
        full_body_pca[[i for i in range(len(clothing_pca)) if full_body_labels[i][1] == gender]][:, 0],
        full_body_pca[[i for i in range(len(clothing_pca)) if full_body_labels[i][1] == gender]][:, 1],
        label=gender
    )

plt.title("PCA Representation of Outfits of Different Gender Categories")
plt.tight_layout()
plt.legend()
plt.show()

for gender in ["mens", "womens"]:
    plt.scatter(
        clothing_tsne[[i for i in range(len(clothing_pca)) if clothing_labels[i][1] == gender]][:, 0],
        clothing_tsne[[i for i in range(len(clothing_pca)) if clothing_labels[i][1] == gender]][:, 1],
        label=gender
    )

plt.title("TSNE Representation of Clothes of Different Clothing Genders")
plt.tight_layout()
plt.legend()
plt.show()

for gender in ["mens", "womens"]:
    plt.scatter(
        full_body_tsne[[i for i in range(len(clothing_pca)) if full_body_labels[i][1] == gender]][:, 0],
        full_body_tsne[[i for i in range(len(clothing_pca)) if full_body_labels[i][1] == gender]][:, 1],
        label=gender
    )

plt.title("TSNE Representation of Outfits of Different Gender Categories")
plt.tight_layout()
plt.legend()
plt.show()

In [ ]:
# Plot types of items
for item in ["shirt", "top", "sweater", "pant", "skirt", "short"]:
    plt.scatter(
        clothing_pca[[i for i in range(len(clothing_pca)) if clothing_labels[i][2] == item]][:, 0],
        clothing_pca[[i for i in range(len(clothing_pca)) if clothing_labels[i][2] == item]][:, 1],
        label=item
    )
    
plt.title("PCA Representation of Clothes of Different Clothing Categories")
plt.tight_layout()
plt.legend()
plt.show()

for item in ["shirt", "top", "sweater", "pant", "skirt", "short"]:
    plt.scatter(
        full_body_pca[[i for i in range(len(clothing_pca)) if full_body_labels[i][2] == item]][:, 0],
        full_body_pca[[i for i in range(len(clothing_pca)) if full_body_labels[i][2] == item]][:, 1],
        label=item
    )
    
plt.title("PCA Representation of Outfits of Different Clothing Categories")
plt.tight_layout()
plt.legend()
plt.show()

for item in ["shirt", "top", "sweater", "pant", "skirt", "short"]:
    plt.scatter(
        clothing_tsne[[i for i in range(len(clothing_pca)) if clothing_labels[i][2] == item]][:, 0],
        clothing_tsne[[i for i in range(len(clothing_pca)) if clothing_labels[i][2] == item]][:, 1],
        label=item
    )
    
plt.title("TSNE Representation of Clothes of Different Clothing Categories")
plt.tight_layout()
plt.legend()
plt.show()

for item in ["shirt", "top", "sweater", "pant", "skirt", "short"]:
    plt.scatter(
        full_body_tsne[[i for i in range(len(clothing_pca)) if full_body_labels[i][2] == item]][:, 0],
        full_body_tsne[[i for i in range(len(clothing_pca)) if full_body_labels[i][2] == item]][:, 1],
        label=item
    )
    
plt.title("TSNE Representation of Outfits of Different Clothing Categories")
plt.tight_layout()
plt.legend()
plt.show()